In [ ]:
import os
os.chdir("..")

In [ ]:
import geopandas as gpd
from shapely import wkt
import polars as pl
import altair as alt
from src.data.data_process import DataClean
ds = DataClean()

In [ ]:
# df = ds.pull_dp05()
df.with_columns(
    under_5_year_male=pl.col("ratio") * pl.col("under_5_year") / (pl.col("ratio") + 100),
    pop_5_9_years_male=pl.col("ratio") * pl.col("pop_5_99") / (pl.col("ratio") + 100),
    pop_10_14_years_male=pl.col("ratio") * pl.col("pop_10_14_years") / (pl.col("ratio") + 100),
    pop_15_19_years_male=pl.col("ratio") * pl.col("pop_15_19_years") / (pl.col("ratio") + 100),
    pop_20_24_years_male=pl.col("ratio") * pl.col("pop_20_24_years") / (pl.col("ratio") + 100),
    pop_25_34_years_male=pl.col("ratio") * pl.col("pop_25_34_years") / (pl.col("ratio") + 100),
    pop_35_44_years_male=pl.col("ratio") * pl.col("pop_35_44_years") / (pl.col("ratio") + 100),
    pop_45_54_years_male=pl.col("ratio") * pl.col("pop_45_54_years") / (pl.col("ratio") + 100),
    pop_55_59_years_male=pl.col("ratio") * pl.col("pop_55_59_years") / (pl.col("ratio") + 100),
    pop_60_64_years_male=pl.col("ratio") * pl.col("pop_60_64_years") / (pl.col("ratio") + 100),
    pop_65_74_years_male=pl.col("ratio") * pl.col("pop_65_74_years") / (pl.col("ratio") + 100),
    pop_75_84_years_male=pl.col("ratio") * pl.col("pop_75_84_years") / (pl.col("ratio") + 100),
    over_85_years_male=pl.col("ratio") * pl.col("over_85_years") / (pl.col("ratio") + 100),
    under_5_year_female=100 * pl.col("under_5_year") / (pl.col("ratio") + 100),
    pop_5_9_years_female=100 * pl.col("pop_5_9_years") / (pl.col("ratio") + 100),
    pop_10_14_years_female=100 * pl.col("pop_10_14_years") / (pl.col("ratio") + 100),
    pop_15_19_years_female=100 * pl.col("pop_15_19_years") / (pl.col("ratio") + 100),
    pop_20_24_years_female=100 * pl.col("pop_20_24_years") / (pl.col("ratio") + 100),
    pop_25_34_years_female=100 * pl.col("pop_25_34_years") / (pl.col("ratio") + 100),
    pop_35_44_years_female=100 * pl.col("pop_35_44_years") / (pl.col("ratio") + 100),
    pop_45_54_years_female=100 * pl.col("pop_45_54_years") / (pl.col("ratio") + 100),
    pop_55_59_years_female=100 * pl.col("pop_55_59_years") / (pl.col("ratio") + 100),
    pop_60_64_years_female=100 * pl.col("pop_60_64_years") / (pl.col("ratio") + 100),
    pop_65_74_years_female=100 * pl.col("pop_65_74_years") / (pl.col("ratio") + 100),
    pop_75_84_years_female=100 * pl.col("pop_75_84_years") / (pl.col("ratio") + 100),
    over_85_years_female=100* pl.col("over_85_years") / (pl.col("ratio") + 100),
    )


In [ ]:
age_df = pl.read_parquet("data/external/age_cal.parquet")
age_df = age_df.with_columns(
    agg=pl.when((pl.col("Edad") < 5)).then(1)
          .when((pl.col("Edad") >= 5) & (pl.col("Edad") < 10)).then(2)
          .when((pl.col("Edad") >= 10) & (pl.col("Edad") < 15)).then(3)
          .when((pl.col("Edad") >= 15) & (pl.col("Edad") < 20)).then(4)
          .when((pl.col("Edad") >= 20) & (pl.col("Edad") < 25)).then(5)
          .when((pl.col("Edad") >= 25) & (pl.col("Edad") < 35)).then(6)
          .when((pl.col("Edad") >= 35) & (pl.col("Edad") < 45)).then(7)
          .when((pl.col("Edad") >= 45) & (pl.col("Edad") < 55)).then(8)
          .when((pl.col("Edad") >= 55) & (pl.col("Edad") < 60)).then(9)
          .when((pl.col("Edad") >= 60) & (pl.col("Edad") < 65)).then(10)
          .when((pl.col("Edad") >= 65) & (pl.col("Edad") < 75)).then(11)
          .when((pl.col("Edad") >= 75) & (pl.col("Edad") < 85)).then(12)
          .when((pl.col("Edad") >= 85)).then(13)
          .otherwise(-1)
    )

age_df = age_df.group_by("agg").agg(
    cal_male=pl.col("Caloriashombre").mean(),
    cal_female=pl.col("Caloriasmujer").mean()

)
age_df.sort("agg")

In [ ]:
tmp = df
for col in df.select(pl.col("^pop_.*$")).columns:
    tmp = tmp.with_columns(
        (pl.col("ratio") * pl.col(col) / (pl.col("ratio") + 100)).alias(col + "_male"),
        (pl.col("ratio") * pl.col(col) / (pl.col("ratio") + 100)).alias(col + "_female"),
    )
tmp